<a href="https://colab.research.google.com/github/djdbgp/DIRpred/blob/master/Dirpred_MSA%2CMSTA_conservation_score_only_by_GPT4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://raw.githubusercontent.com/djdbgp/DIRpred/refs/heads/master/src/utils.py

--2024-12-29 12:18:54--  https://raw.githubusercontent.com/djdbgp/DIRpred/refs/heads/master/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40726 (40K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  39.77K  --.-KB/s    in 0.05s   

2024-12-29 12:18:55 (753 KB/s) - ‘utils.py’ saved [40726/40726]



In [4]:
pip install biopython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 28.1 MB/s eta 0:00:00


In [18]:
from google.colab import files

# 생성된 파일 다운로드
files.download("./output/msa_scores.csv")
files.download("./output/msta_scores.csv")


FileNotFoundError: Cannot find file: ./output/msa_scores.csv

In [9]:
import os

# 출력 디렉터리 확인
output_path = "./output"
print("Output directory exists:", os.path.exists(output_path))
print("Contents of output directory:", os.listdir(output_path) if os.path.exists(output_path) else "Directory does not exist")


Output directory exists: False
Contents of output directory: Directory does not exist


In [11]:
if not os.path.exists(output_path):
    os.makedirs(output_path)
    print(f"Directory created at {output_path}")


Directory created at ./output


In [52]:
import utils as dc
import os
import pandas as pd
from google.colab import files

def upload_files():
    """
    Google Colab에서 파일 업로드
    """
    uploaded = files.upload()
    return uploaded

def read_input_files(msa_file, msta_file, reference):
    """
    MSA와 MSTA 파일을 읽고 객체로 변환합니다.
    """
    msa = dc.MSA("MSA", reference=reference)
    msa.msa = dc.create_msa(open(msa_file))

    msta = dc.MSA("MSTA", reference=reference)
    msta.msa = dc.create_msa(open(msta_file))

    return msa, msta

def filter_positions(df):
    """
    MSA 또는 MSTA 데이터에서 X 값을 가진 위치를 제거하고 메싸이오닌(M)이 1번 위치가 되도록 조정합니다.
    """
    # X 값과 숫자가 결합된 형태도 제거
    df["Position"] = df["Position"].astype(str)  # 문자열로 변환
    filtered_df = df[~df["Position"].str.contains(r"X", na=False)]

    # 메싸이오닌(M)을 기준으로 위치 재조정
    try:
        methionine_index = filtered_df[filtered_df["Position"].str.contains("M")].index.min()
        if pd.notna(methionine_index):
            filtered_df = filtered_df.reset_index(drop=True)  # 인덱스 재설정
            filtered_df["Position"] = filtered_df.index + 1  # 1부터 시작하도록 설정
        else:
            print("Warning: Methionine (M) not found. Retaining original positions.")
    except Exception as e:
        print(f"Error during Methionine adjustment: {e}")
        return df

    print(f"Filtered and adjusted DataFrame: {filtered_df.shape[0]} positions retained")
    return filtered_df

def calculate_msa_msta(msa, msta, output_path):
    """
    MSA와 MSTA 점수를 계산하고 저장합니다.
    """
    msa_scores = msa.get_cons_scores("id")
    msta_scores = msta.get_cons_scores("id")

    msa_df = pd.DataFrame({"Position": msa.get_referenced_pos(), "Score": msa_scores})
    msta_df = pd.DataFrame({"Position": msta.get_referenced_pos(), "Score": msta_scores})

    # MSA와 MSTA 데이터를 각각 필터링
    msa_df = filter_positions(msa_df)
    msta_df = filter_positions(msta_df)

    # 결과를 CSV로 저장
    msa_output_path = os.path.join(output_path, "msa_scores.csv")
    msta_output_path = os.path.join(output_path, "msta_scores.csv")

    msa_df.to_csv(msa_output_path, index=False)
    msta_df.to_csv(msta_output_path, index=False)

    print(f"MSA scores saved to {msa_output_path}")
    print(f"MSTA scores saved to {msta_output_path}")

    return msa_df, msta_df

def calculate_combined_conservation(msa_df, msta_df, output_path):
    """
    MSA와 MSTA 점수를 결합하여 보존성 점수를 계산합니다.
    """
    # MSA와 MSTA 데이터를 Position을 기준으로 병합
    combined_df = pd.merge(msa_df, msta_df, on="Position", suffixes=("_MSA", "_MSTA"), how="inner")

    # 결합된 Conservation Score 계산 (가중 평균 예제)
    combined_df["Combined_Score"] = (combined_df["Score_MSA"] + combined_df["Score_MSTA"]) / 2

    # 결과를 CSV로 저장
    combined_output_path = os.path.join(output_path, "combined_scores.csv")
    combined_df.to_csv(combined_output_path, index=False)

    print(f"Combined conservation scores saved to {combined_output_path}")

def main():
    # 파일 업로드
    uploaded = upload_files()

    # 업로드된 파일 이름 가져오기
    msa_file = list(uploaded.keys())[0]  # 첫 번째 업로드 파일을 MSA로 사용
    msta_file = list(uploaded.keys())[1]  # 두 번째 업로드 파일을 MSTA로 사용

    output_path = "./output"  # 출력 경로

    # 참조 서열 선택
    msa = dc.create_msa(open(msa_file))
    reference_options = msa.index.tolist()
    print("Available references:", reference_options)

    # 사용자 입력과 실제 참조 이름 자동 매칭
    user_input = input(f"Choose a reference from the following (partial match allowed):\n")
    reference = next((name for name in reference_options if user_input.lower() in name.lower()), None)

    if reference is None:
        print("Error: No matching reference found. Please try again.")
        return

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    msa, msta = read_input_files(msa_file, msta_file, reference)
    msa_df, msta_df = calculate_msa_msta(msa, msta, output_path)
    calculate_combined_conservation(msa_df, msta_df, output_path)

if __name__ == "__main__":
    main()


Saving 20 sequences Alignment.fasta to 20 sequences Alignment.fasta
Saving foldmason_aa.fa to foldmason_aa.fa
Available references: ['aave2166', 'xopj2', 'hopz5', 'aave2708', 'xopj1', 'xopj3', 'xopj4', 'hopz1a', 'hopz1b', 'hopz2', 'hopz4', 'ripp1', 'eop1', 'hopz3', 'hopz1c', 'ripj', 'ripp2', 'xopj5', 'xopj6', 'ripk']
Choose a reference from the following (partial match allowed):
hopz5
Filtered and adjusted DataFrame: 346 positions retained
Filtered and adjusted DataFrame: 346 positions retained
MSA scores saved to ./output/msa_scores.csv
MSTA scores saved to ./output/msta_scores.csv
Combined conservation scores saved to ./output/combined_scores.csv


In [53]:
import utils as dc
import os
import pandas as pd
import numpy as np
from google.colab import files
from scipy.stats import entropy

def upload_files():
    """
    Google Colab에서 파일 업로드
    """
    uploaded = files.upload()
    return uploaded

def read_input_files(msa_file, msta_file, reference):
    """
    MSA와 MSTA 파일을 읽고 객체로 변환합니다.
    """
    msa = dc.MSA("MSA", reference=reference)
    msa.msa = dc.create_msa(open(msa_file))

    msta = dc.MSA("MSTA", reference=reference)
    msta.msa = dc.create_msa(open(msta_file))

    return msa, msta

def filter_positions(df):
    """
    MSA 또는 MSTA 데이터에서 X 값을 가진 위치를 제거하고 메싸이오닌(M)이 1번 위치가 되도록 조정합니다.
    """
    # X 값과 숫자가 결합된 형태도 제거
    df["Position"] = df["Position"].astype(str)  # 문자열로 변환
    filtered_df = df[~df["Position"].str.contains(r"X", na=False)]

    # 메싸이오닌(M)을 기준으로 위치 재조정
    try:
        methionine_index = filtered_df[filtered_df["Position"].str.contains("M")].index.min()
        if pd.notna(methionine_index):
            filtered_df = filtered_df.reset_index(drop=True)  # 인덱스 재설정
            filtered_df["Position"] = filtered_df.index + 1  # 1부터 시작하도록 설정
        else:
            print("Warning: Methionine (M) not found. Retaining original positions.")
    except Exception as e:
        print(f"Error during Methionine adjustment: {e}")
        return df

    print(f"Filtered and adjusted DataFrame: {filtered_df.shape[0]} positions retained")
    return filtered_df

def calculate_al2co_conservation(msa):
    """
    AL2CO 방식으로 보존성 점수를 계산합니다 (Shannon 엔트로피 기반).
    """
    msa_array = np.array(msa.msa)
    num_positions = msa_array.shape[1]  # 서열의 길이
    conservation_scores = []

    for i in range(num_positions):
        column = msa_array[:, i]  # 각 위치의 서열 열 추출
        unique, counts = np.unique(column, return_counts=True)  # 잔기 빈도 계산
        probabilities = counts / counts.sum()  # 확률 분포 생성
        cons_score = 1 - entropy(probabilities, base=2)  # Shannon 엔트로피 계산 후 보존성 변환
        conservation_scores.append(cons_score)

    return conservation_scores

def calculate_msa_msta(msa, msta, output_path):
    """
    MSA와 MSTA 점수를 계산하고 저장합니다.
    """
    msa_scores = msa.get_cons_scores("id")
    msta_scores = msta.get_cons_scores("id")

    msa_df = pd.DataFrame({"Position": msa.get_referenced_pos(), "Score": msa_scores})
    msta_df = pd.DataFrame({"Position": msta.get_referenced_pos(), "Score": msta_scores})

    # MSA와 MSTA 데이터를 각각 필터링
    msa_df = filter_positions(msa_df)
    msta_df = filter_positions(msta_df)

    # AL2CO 방식으로 추가 점수 계산
    al2co_scores = calculate_al2co_conservation(msa)
    msa_df["AL2CO_Score"] = al2co_scores[:len(msa_df)]  # AL2CO 점수를 추가

    # 결과를 CSV로 저장
    msa_output_path = os.path.join(output_path, "msa_scores.csv")
    msta_output_path = os.path.join(output_path, "msta_scores.csv")

    msa_df.to_csv(msa_output_path, index=False)
    msta_df.to_csv(msta_output_path, index=False)

    print(f"MSA scores saved to {msa_output_path}")
    print(f"MSTA scores saved to {msta_output_path}")

    return msa_df, msta_df

def calculate_combined_conservation(msa_df, msta_df, output_path):
    """
    MSA와 MSTA 점수를 결합하여 보존성 점수를 계산합니다.
    """
    # MSA와 MSTA 데이터를 Position을 기준으로 병합
    combined_df = pd.merge(msa_df, msta_df, on="Position", suffixes=("_MSA", "_MSTA"), how="inner")

    # 결합된 Conservation Score 계산 (가중 평균 예제)
    combined_df["Combined_Score"] = (combined_df["Score_MSA"] + combined_df["Score_MSTA"] + combined_df["AL2CO_Score"]) / 3

    # 결과를 CSV로 저장
    combined_output_path = os.path.join(output_path, "combined_scores.csv")
    combined_df.to_csv(combined_output_path, index=False)

    print(f"Combined conservation scores saved to {combined_output_path}")

def main():
    # 파일 업로드
    uploaded = upload_files()

    # 업로드된 파일 이름 가져오기
    msa_file = list(uploaded.keys())[0]  # 첫 번째 업로드 파일을 MSA로 사용
    msta_file = list(uploaded.keys())[1]  # 두 번째 업로드 파일을 MSTA로 사용

    output_path = "./output"  # 출력 경로

    # 참조 서열 선택
    msa = dc.create_msa(open(msa_file))
    reference_options = msa.index.tolist()
    print("Available references:", reference_options)

    # 사용자 입력과 실제 참조 이름 자동 매칭
    user_input = input(f"Choose a reference from the following (partial match allowed):\n")
    reference = next((name for name in reference_options if user_input.lower() in name.lower()), None)

    if reference is None:
        print("Error: No matching reference found. Please try again.")
        return

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    msa, msta = read_input_files(msa_file, msta_file, reference)
    msa_df, msta_df = calculate_msa_msta(msa, msta, output_path)
    calculate_combined_conservation(msa_df, msta_df, output_path)

if __name__ == "__main__":
    main()


Saving 20 sequences Alignment.fasta to 20 sequences Alignment.fasta
Saving foldmason_aa.fa to foldmason_aa.fa
Available references: ['aave2166', 'xopj2', 'hopz5', 'aave2708', 'xopj1', 'xopj3', 'xopj4', 'hopz1a', 'hopz1b', 'hopz2', 'hopz4', 'ripp1', 'eop1', 'hopz3', 'hopz1c', 'ripj', 'ripp2', 'xopj5', 'xopj6', 'ripk']
Choose a reference from the following (partial match allowed):
hopz5
Filtered and adjusted DataFrame: 346 positions retained
Filtered and adjusted DataFrame: 346 positions retained
MSA scores saved to ./output/msa_scores.csv
MSTA scores saved to ./output/msta_scores.csv
Combined conservation scores saved to ./output/combined_scores.csv
